In [6]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [5]:
! pip install plotly

     |████████████████████████████████| 11.5MB 3.9MB/s eta 0:00:01    |██▏                             | 757kB 3.9MB/s eta 0:00:03
  Stored in directory: /Users/sailyjog/Library/Caches/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [7]:
full_table = pd.read_csv('/Users/sailyjog/Desktop/Spring 2020/Data Engineering/Project/covid_19_clean_complete.csv', 
                         parse_dates=['Date'])
full_table.sample(6)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
5152,NaN,Israel,31.0000,35.0000,2020-02-10,0,0,0
14437,NaN,Peru,-9.1900,-75.0152,2020-03-16,86,0,0
15807,Diamond Princess,Canada,0.0000,0.0000,2020-03-21,0,0,0
6027,Channel Islands,United Kingdom,49.3723,-2.3644,2020-02-13,0,0,0
26329,NaN,Senegal,14.4974,-14.4524,2020-04-30,933,9,334
19064,Guizhou,China,26.8154,106.8748,2020-04-03,146,2,144


In [8]:
full_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27984 entries, 0 to 27983
Data columns (total 8 columns):
Province/State    8480 non-null object
Country/Region    27984 non-null object
Lat               27984 non-null float64
Long              27984 non-null float64
Date              27984 non-null datetime64[ns]
Confirmed         27984 non-null int64
Deaths            27984 non-null int64
Recovered         27984 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 1.7+ MB


In [9]:
full_table.isna().sum()

Province/State    19504
Country/Region        0
Lat                   0
Long                  0
Date                  0
Confirmed             0
Deaths                0
Recovered             0
dtype: int64

In [10]:
full_table.shape

(27984, 8)

In [11]:
ship_rows = full_table['Province/State'].str.contains('Grand Princess') | full_table['Province/State'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('MS Zaandam')

# ship
ship = full_table[ship_rows]

# full table 
full_table = full_table[~(ship_rows)]

# Latest cases from the ships
ship_latest = ship[ship['Date']==max(ship['Date'])]

In [12]:
ship_latest.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
27757,Grand Princess,Canada,37.6489,-122.6655,2020-05-06,13,0,0
27808,NaN,Diamond Princess,0.0000,0.0000,2020-05-06,712,13,645
27951,Diamond Princess,Canada,0.0000,0.0000,2020-05-06,1,1,0
27971,NaN,MS Zaandam,0.0000,0.0000,2020-05-06,9,2,0


In [13]:
ship_latest.shape

(4, 8)

In [14]:
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']

In [15]:
full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0,0,0
1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0,0,0
2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0,0,0
3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0,0,0
4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0,0,0


In [16]:
full_table['Country/Region'] = full_table['Country/Region'].replace('Mainland China', 'China')

In [17]:
full_table[['Province/State']] = full_table[['Province/State']].fillna('')
full_table[['Confirmed', 'Deaths', 'Recovered', 'Active']] = full_table[['Confirmed', 'Deaths', 'Recovered', 'Active']].fillna(0)

In [19]:
full_table['Recovered'] = full_table['Recovered'].astype(int)

In [20]:
full_table.sample(6)


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
24457,,Netherlands,52.1326,5.2913,2020-04-23,35729,4177,0,31552
19941,,Kazakhstan,48.0196,66.9237,2020-04-06,662,6,46,610
11963,,Congo (Brazzaville),-4.0383,21.7587,2020-03-07,0,0,0,0
3741,Saskatchewan,Canada,52.9399,-106.4509,2020-02-05,0,0,0,0
12920,Anguilla,United Kingdom,18.2206,-63.0686,2020-03-10,0,0,0,0
25471,,Holy See,41.9029,12.4534,2020-04-27,9,0,2,7


In [21]:
full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()

In [22]:
# new cases ======================================================
temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()

In [23]:
mask = temp['Country/Region'] != temp['Country/Region'].shift(1)

In [24]:
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan

In [25]:
temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']

In [26]:
temp.head()

,Country/Region,Date,New cases,New deaths,New recovered
0,Afghanistan,2020-01-22,NaN,NaN,NaN
1,Afghanistan,2020-01-23,0.0,0.0,0.0
2,Afghanistan,2020-01-24,0.0,0.0,0.0
3,Afghanistan,2020-01-25,0.0,0.0,0.0
4,Afghanistan,2020-01-26,0.0,0.0,0.0


In [27]:
full_grouped = pd.merge(full_grouped, temp, on=['Country/Region', 'Date'])

In [28]:
full_grouped = full_grouped.fillna(0)

In [29]:
cols = ['New cases', 'New deaths', 'New recovered']
full_grouped[cols] = full_grouped[cols].astype('int')

In [30]:
full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)

full_grouped.head()

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
0,2020-01-22,Afghanistan,0,0,0,0,0,0,0
1,2020-01-22,Albania,0,0,0,0,0,0,0
2,2020-01-22,Algeria,0,0,0,0,0,0,0
3,2020-01-22,Andorra,0,0,0,0,0,0,0
4,2020-01-22,Angola,0,0,0,0,0,0,0


In [31]:
day_wise = full_grouped.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active', 'New cases'].sum().reset_index()

In [32]:
day_wise['No. of countries'] = full_grouped[full_grouped['Confirmed']!=0].groupby('Date')['Country/Region'].unique().apply(len).values

In [34]:
day_wise.head()

,Date,Confirmed,Deaths,Recovered,Active,New cases,No. of countries
0,2020-01-22,555,17,28,510,0,6
1,2020-01-23,654,18,30,606,99,8
2,2020-01-24,941,26,35,880,287,9
3,2020-01-25,1434,42,38,1354,493,11
4,2020-01-26,2118,56,51,2011,684,13


In [35]:
country_wise = full_grouped[full_grouped['Date']==max(full_grouped['Date'])].reset_index(drop=True).drop('Date', axis=1)

In [36]:
country_wise = country_wise.groupby('Country/Region')['Confirmed', 'Deaths', 'Recovered', 'Active', 'New cases'].sum().reset_index()

In [37]:
country_wise.head()

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases
0,Afghanistan,3392,104,458,2830,168
1,Albania,832,31,595,206,12
2,Algeria,4997,476,2197,2324,159
3,Andorra,751,46,521,184,0
4,Angola,36,2,11,23,0


In [39]:
import plotly.express as px

In [61]:
temp = full_table.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()
temp = temp.melt(id_vars="Date", value_vars=['Recovered', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')
temp.head()

fig = px.area(temp, x="Date", y="Count", color='Case', height=600,
             title='Cases over time')
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

In [46]:
temp = full_table.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()
temp = temp.melt(id_vars="Date", value_vars=['Recovered', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')
temp.head()

fig = px.area(temp, x="Date", y="Count", color='Case', height=600,
             title='Cases over time')
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

In [49]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [53]:
fig_c = px.bar(day_wise, x="Date", y="Confirmed")
fig_d = px.bar(day_wise, x="Date", y="Deaths")

fig = make_subplots(rows=1, cols=2, shared_xaxes=False, horizontal_spacing=0.1,
                    subplot_titles=('Confirmed cases', 'Deaths reported'))

fig.add_trace(fig_c['data'][0], row=1, col=1)
fig.add_trace(fig_d['data'][0], row=1, col=2)

fig.update_layout(height=480)
fig.show()

In [54]:
fig_c = px.bar(day_wise, x="Date", y="New cases")
fig_d = px.bar(day_wise, x="Date", y="No. of countries")

fig = make_subplots(rows=1, cols=2, shared_xaxes=False, horizontal_spacing=0.1,
                    subplot_titles=('No. of new cases everyday', 'No. of countries'))

fig.add_trace(fig_c['data'][0], row=1, col=1)
fig.add_trace(fig_d['data'][0], row=1, col=2)

fig.update_layout(height=480)
fig.show()

In [59]:
fig_c = px.bar(country_wise.sort_values('Confirmed').tail(15), x="Confirmed", y="Country/Region", 
               text='Confirmed', orientation='h')
fig_d = px.bar(country_wise.sort_values('Deaths').tail(15), x="Deaths", y="Country/Region", 
               text='Deaths', orientation='h',color_discrete_sequence = ['red'])

# recovered - active
fig_r = px.bar(country_wise.sort_values('Recovered').tail(15), x="Recovered", y="Country/Region", 
               text='Recovered', orientation='h',color_discrete_sequence = ['green'])
fig_a = px.bar(country_wise.sort_values('Active').tail(15), x="Active", y="Country/Region", 
               text='Active', orientation='h', color_discrete_sequence = ['yellow'])
fig_c.show()
fig_d.show()
fig_r.show()
fig_a.show()